In [139]:
import yfinance as yf
import time
import pandas as pd
import json
import numpy as np
import xgboost
import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from xgboost import XGBClassifier

In [153]:
import optuna
import xgboost as xgb
from sklearn.model_selection import train_test_split, cross_val_score
# from sklearn.datasets import load_boston
from sklearn.metrics import mean_squared_error

In [2]:
hedging_pnl = pd.read_csv('Options constructor dummy - raw_hedging_pnls.csv')
options_pnl = pd.read_csv('Options constructor dummy - raw_options_pnls.csv')

hedging_pnl['0'] = pd.to_datetime(hedging_pnl['0'])
options_pnl['0'] = pd.to_datetime(options_pnl['0'])

options_pnl = options_pnl.set_index('0').astype(float)
hedging_pnl = hedging_pnl.set_index('0').astype(float)

base_features = pd.read_csv('new_features.csv')
base_features['Unnamed: 0'] = pd.to_datetime(base_features['Unnamed: 0'])
base_features = base_features.set_index('Unnamed: 0')
base_features = base_features.resample('W-SUN').sum()

strats = list(hedging_pnl.columns)

In [3]:
strats_data = {}
for i in strats:
    strat_info = (options_pnl[i]+hedging_pnl[i])
    weekly_pnl = strat_info.resample('W-SUN').sum()

    strats_data.update({i:weekly_pnl})

In [115]:
for strat in strats:
    
    str_data = pd.DataFrame()
    str_data['value'] = strats_data[strats[100]]
    str_data['target'] = str_data['value'].shift(-1)
    
    
    df = pd.merge(str_data, base_features, how = 'inner', right_index = True, left_index = True).dropna()
    df['target'] = np.where(df['target']>=0,1,0)

    features = df.drop('target', axis = 1)
    target = df['target']

    break

In [117]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3)

In [119]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.fit_transform(y_test)

In [121]:
y_test

array([1, 0, 1, 1, 1, 1], dtype=int64)

In [123]:
xgb = XGBClassifier(
                   learning_rate = 0.02,
                   n_estimators = 100,
                   max_depth = 3,
                   min_child_weight = 1,
                   subsample = 0.75,
                   colsample_bytree = 0.6,
                   gamma = 0)


# Fitting the model on the training data
xgb.fit(X_train, y_train)

xgb.save_model('xgb_last.bin')  # or 'model_name.model'


C:\Users\Alexander\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:42:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\c_api\c_api.cc:1374: Saving model in the UBJSON format as default.  You can use file extension: `json`, `ubj` or `deprecated` to choose between formats.
  warnings.warn(smsg, UserWarning)


In [147]:
def objective(trial):
    param = {
        'tree_method': 'hist',  # Fast method for large datasets
        'objective': 'reg:squarederror',
        'eval_metric': 'rmse',
        'booster': trial.suggest_categorical('booster', ['gbtree', 'gblinear', 'dart']),
        'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
        'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
        'subsample': trial.suggest_uniform('subsample', 0.4, 1.0),
        'colsample_bytree': trial.suggest_uniform('colsample_bytree', 0.4, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 9),
        'n_estimators': trial.suggest_int('n_estimators', 100, 1000),
        'learning_rate': trial.suggest_loguniform('learning_rate', 1e-8, 1.0),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 10)
    }

    model = XGBClassifier(**param)
    model.fit(X_train, y_train)

    preds = model.predict(X_test)

    rmse = mean_squared_error(y_test, preds, squared=False)

    return rmse

In [155]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=100)

print("Best hyperparameters: ", study.best_params)

[I 2024-08-09 15:47:17,309] A new study created in memory with name: no-name-677d5822-424c-45f8-811c-4a2c3a9b91bd
C:\Users\Alexander\AppData\Local\Temp\ipykernel_17184\362278896.py:8: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'lambda': trial.suggest_loguniform('lambda', 1e-8, 1.0),
C:\Users\Alexander\AppData\Local\Temp\ipykernel_17184\362278896.py:9: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  'alpha': trial.suggest_loguniform('alpha', 1e-8, 1.0),
C:\Users\Alexander\AppData\Local\Temp\ipykernel_17184\362278896.py:10: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/t

KeyboardInterrupt: 

In [161]:
best_params = study.best_params

model = XGBClassifier(**best_params)
model.fit(X_train, y_train)

preds = model.predict(X_test)
rmse = mean_squared_error(y_test, preds, squared=False)
print(f"RMSE: {rmse}")

RMSE: 0.37273465271134126


C:\Users\Alexander\anaconda3\Lib\site-packages\xgboost\core.py:158: UserWarning: [15:48:29] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0015a694724fa8361-1\xgboost\xgboost-ci-windows\src\learner.cc:740: 
Parameters: { "colsample_bytree", "max_depth", "min_child_weight", "subsample" } are not used.

  warnings.warn(smsg, UserWarning)
C:\Users\Alexander\anaconda3\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
